In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import BertTokenizer, TFBertForSequenceClassification

In [8]:
df = pd.read_csv("D:/datasets/tweets.csv") 
df = df[['text', 'target']]  
df.dropna(inplace=True)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   5%|4         | 21.0M/440M [00:00<?, ?B/s]

In [5]:
def tokenize_function(texts):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )

X_tokenized = tokenize_function(df['text'])

In [6]:
labels = tf.convert_to_tensor(df['target'].values)
dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_tokenized),
    labels
))

In [7]:
dataset = dataset.shuffle(1000)
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size).batch(8)
test_dataset = dataset.skip(train_size).batch(8)

In [8]:
config = BertConfig.from_pretrained("bert-base-uncased", num_labels=2)
model = TFBertForSequenceClassification(config)

NameError: name 'BertConfig' is not defined

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=3,
    batch_size=8,
    validation_split=0.1,
    verbose=2
)

In [ ]:
logits = model.predict(X_test).logits
y_pred = tf.argmax(logits, axis=1).numpy()
y_true = y_test.numpy()

In [ ]:
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not Disaster", "Disaster"])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.grid(False)
plt.show()